# 01. Carga de datos y EDA inicial
- Cargar CSV original
- Número de filas y columnas
- Existencia de duplicados
- Tipos de variables
- Valores nulos por columna

In [22]:
# Core

import requests
import pandas as pd
# Configuración para que muestre todas las columnas al completo
pd.set_option('display.max_columns', None)
# Configuración para que no muestre la notación científica
# pd.set_option('display.float_format', '{:,.0f}'.format)

import numpy as np
import re

# Configuración

import warnings
warnings.filterwarnings("ignore")

# Utilidades

# para convertir str en números
from word2number import w2n

In [3]:

def carga_eda(csv):
    """ 
    Función para leer csv, convertir a df y hacer una primera exploración.
    Igualar a variable con el nombre que quieres dar a DataFrame
    """
    
    try:
        # Convertir el csv a DataFrame
        df = pd.read_csv(f"../data/{csv}.csv")        

        # Muestro las primeras filas
        display(df.head())

        # Muestro las últimas filas
        display(df.tail())

        # Muestro las dimensiones del dataframe
        print(f"-----\n\nEl DataFrame tiene {df.shape[0]} filas y {df.shape[1]} columnas.\n-----")

        # Consulto si hay filas duplicadas
        print(f"\nEl número de filas duplicadas es {df.duplicated().sum()}\n-----")

        # Muestro el tipo de dato y si hay nulos por cada columna
        print("\nInformación del DataFrame:")
        df.info()

        # Muestro el porcentaje de nulos por variable
        print("\nPorcentaje de nulos:")
        display(round(df.isnull().mean() * 100, 2))

        # Muestro las estadísticas de columnas numéricas
        print("-----\n\nEstadísticas descriptivas:")
        display(df.describe(include="all").T)

        # Me devuelve un df que tendré que igualar a una variable
        return df  
                

    # Excepciones en caso de no encontrar el archivo o de que haya un error
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo '../data/{csv}.csv'.")
        return None  
    
    except Exception as e:
        print(f"Error: {e}")
        return None 

In [4]:
df_movies = carga_eda("raw_movies_dataset")

,Title,Genre,Year,Budget,Revenue,IMDB_Rating
0,The Last Journey,Drama,2020,11000000,3.404969e+07,4.9
1,Infinite Dreams,Drama,2004,178000000,5.504360e+08,6.6
2,Code of Shadows,Romance,2018,13000000,7.855773e+06,4.7
3,Ocean Whisper,Action,2016,148000000,1.769979e+08,7.2
4,Forgotten Realm,Romance,2017,61000000,1.264061e+08,NaN


,Title,Genre,Year,Budget,Revenue,IMDB_Rating
17,Shattered Skies,NaN,2008,59000000,2.180626e+08,6.7
18,Eternal Voyage,Fantasy,2022,59000000,NaN,5.8
19,Mystic River,NaN,2020,41000000,5.851030e+07,4.9
20,The Last Journey,Drama,2020,11000000,3.404969e+07,4.9
21,Shattered Skies,NaN,2008,59000000,2.180626e+08,6.7


-----

El DataFrame tiene 22 filas y 6 columnas.
-----

El número de filas duplicadas es 2
-----

Información del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Title        22 non-null     object 
 1   Genre        18 non-null     object 
 2   Year         22 non-null     object 
 3   Budget       22 non-null     object 
 4   Revenue      19 non-null     float64
 5   IMDB_Rating  18 non-null     float64
dtypes: float64(2), object(4)
memory usage: 1.2+ KB

Porcentaje de nulos:


Title           0.00
Genre          18.18
Year            0.00
Budget          0.00
Revenue        13.64
IMDB_Rating    18.18
dtype: float64

-----

Estadísticas descriptivas:


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Title,22,20,The Last Journey,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Genre,18,7,Drama,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Year,22,15,2020,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Budget,22,19,59000000,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Revenue,19.0,NaN,NaN,NaN,173447915.703216,136688546.729653,7855773.484432,59656929.564439,150236576.310303,221459088.660491,550436004.927962
IMDB_Rating,18.0,NaN,NaN,NaN,5.883333,1.487299,3.9,4.9,5.55,6.7,9.1


# 02. Visualizaciones exploratorias


# 03. Limpieza de variables sin nulos
- Conversión de tipos (Year a int o fecha, 
- Revenue y Budget a números)
- Estándar de strings (Title, Genre, etc.)
- Eliminación de duplicados

In [10]:
def text_to_num(year):
    """
    Limpiar y convertir a números str sin errores ortográficos ni de puntuación: 
        - Si el valor es un str, lo normaliza dejando todo en minusculas y quitando espacios en los extremos.
        - Después lo convierte a número con la librería w2n, método word_to_num
        - Si no puede devuelve el valor ya normalizado
        - Si no es un str, devuelve el valor original
    """
    # Si es str
    if isinstance(year, str):
        # Normalizamos el str
        year_limpio = year.strip().lower()
        try:
            return w2n.word_to_num(year_limpio)
        except:
            return year_limpio
    # Si no es str, devuelve el mismo valor   
    else: 
        return year

In [ ]:
def clean_budget(num):
        # Quitar espacios al inicio/final
        num = num.strip() 

        # Buscar si hay "M" o "m":
        if re.search(r"[Mm]", num):

            # Quitar "M" o "m" y convertir a int y millones
            clean_num = re.sub(r"[^\d\.]", "", num)
            try:
                return int(clean_num) * 1_000_000
            except:
                return np.nan

        # Buscar si hay "K" o "k":
        if re.search(r"[Kk]", num):

            # Quitar "K" o "k" y convertir a int y miles
            clean_num = re.sub(r"[^\d\.]", "", num)
            try:
                return int(clean_num) * 1_000
            except:
                return np.nan   
            
        # Si no encuentra solo convierte a Int
        else:
            
            try:
                return int(num)
            except:
                return np.nan

## 03.01 Limpieza Year

In [9]:
# Compruebo los valores que se encuentran dentro de la variable "Year"
# Busco una librería para convertir str a numero, siempre y cuando no hayan errores ortográficos ni de puntuación.

df_movies["Year"].value_counts()

Year
2020            3
2011            2
2007            2
2022            2
2004            1
2018            1
2016            1
2017            1
2000            1
2006            1
2003            1
2019            1
Two Thousand    1
2001            1
2008            1
Name: count, dtype: int64

In [11]:
# Aplico la función a la variable "Year" del DataFrame

df_movies["Year"] = df_movies["Year"].apply(text_to_num)

In [12]:
# Convierto str a datetime y luego a year.

df_movies["Year"] = pd.to_datetime(df_movies["Year"], format="%Y").dt.year

In [14]:
# Compruebo el resultado, ordenando el output por año

df_movies["Year"].value_counts().sort_index()

Year
2000    2
2001    1
2003    1
2004    1
2006    1
2007    2
2008    1
2011    2
2016    1
2017    1
2018    1
2019    1
2020    3
2022    2
Name: count, dtype: int64

## 03.02 Limpieza Budget

In [15]:
# Entro con la última variable en la que no hay valores nulos, veo todos los valores que encuentro dentro
# Este paso no es escalable, pensaría en hacer un .sample() o un .head() o un .tail()

df_movies["Budget"].value_counts()

Budget
59000000     2
11000000     1
202000000    1
215000000    1
180000000    1
123000000    1
160000000    1
99000000     1
102000000    1
152000000    1
178000000    1
25000000     1
211000000    1
31000000     1
80M          1
61000000     1
148000000    1
13000000     1
41000000     1
Name: count, dtype: int64

In [18]:
# Aplico la función 

df_movies["Budget"] = df_movies["Budget"].apply(clean_budget)

In [17]:
print("\nPorcentaje de nulos:")
display(round(df_movies.isnull().mean() * 100, 2))


Porcentaje de nulos:


Title           0.0
Genre          15.0
Year            0.0
Budget          0.0
Revenue        15.0
IMDB_Rating    20.0
dtype: float64

## 03.03 Duplicados

In [5]:
# Miro los valores que se encuentran dentro de la variable "Title":
#   - encuentro los que están duplicados
#   - compruebo si es necesaria la normalización del texto, NO

df_movies["Title"].value_counts()

Title
The Last Journey        2
Shattered Skies         2
Infinite Dreams         1
Eternal Voyage          1
Broken Destiny          1
Beyond the Stars        1
The Lost Signal         1
Crimson Night           1
Quantum Hearts          1
Solar Storm             1
Iron Legacy             1
Whispers in the Dark    1
Digital Frontier        1
Echoes of Tomorrow      1
Silent Horizon          1
Neon City               1
Forgotten Realm         1
Ocean Whisper           1
Code of Shadows         1
Mystic River            1
Name: count, dtype: int64

In [6]:
# Miro las filas duplicadas con sus originales, compruebo que son realmente duplicados.
# Se puede añadir un .head() en caso de querer escalar a un dataset más grande, para ver los primeros duplicados y empezar a analizar por ahí.

df_movies[df_movies.duplicated(keep=False)].sort_values("Title")

,Title,Genre,Year,Budget,Revenue,IMDB_Rating
17,Shattered Skies,NaN,2008,59000000,2.180626e+08,6.7
21,Shattered Skies,NaN,2008,59000000,2.180626e+08,6.7
0,The Last Journey,Drama,2020,11000000,3.404969e+07,4.9
20,The Last Journey,Drama,2020,11000000,3.404969e+07,4.9


In [7]:
# Elimino los duplicados.

df_movies.drop_duplicates(inplace=True)

In [8]:
# Compruebo las nuevas dimensiones una vez eliminados los duplicados.

df_movies.shape

(20, 6)

In [13]:
# Reviso la info de las columnas hasta el momento

df_movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20 entries, 0 to 19
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Title        20 non-null     object 
 1   Genre        17 non-null     object 
 2   Year         20 non-null     int32  
 3   Budget       20 non-null     object 
 4   Revenue      17 non-null     float64
 5   IMDB_Rating  16 non-null     float64
dtypes: float64(2), int32(1), object(3)
memory usage: 1.0+ KB


# 04. Limpieza y relleno de nulos
- Variables con nulos: usar función OMDb para rellenar IMDB_Rating, Revenue, Genre
- Imputación adicional si quedan nulos

In [24]:
API_KEY = "XXX"

"En este bloque controlo: 
- que solo se rellenen las columnas que están vacías realmente en el DataFrame y 
- que la API realmente devuelva ese dato,
- que imprima cuando no encuentre la película
- que imprima los datos que ha rellenado

Así no se sobreescribe información que ya tenía limpia y evito errores si en la respuesta de la API no está la key. 
Actualización es segura y consistente."

In [25]:
# CORRECTA MEJOR MÁS EFICIENTE 
def fill_omdb(row, columns_to_fill=["IMDB_Rating", "Revenue", "Genre"]):
    """
    Toma una fila de un DataFrame, comprueba si tiene NaN en las columnas especificadas,
    y si es así consulta OMDb por el título y devuelve la fila con los valores rellenados.
    """
    # Si no falta nada, devuelve la fila tal cual
    if not row[columns_to_fill].isnull().any():
        return row

    title = row["Title"]

    url = f"http://www.omdbapi.com/?t={title}&apikey={API_KEY}"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()

        if data.get("Response") == "True":
            if pd.isna(row["IMDB_Rating"]) and "imdbRating" in data:
                if data["imdbRating"] != "N/A":
                    row["IMDB_Rating"] = data["imdbRating"]
                    print(f"{title} -> IMDB_Rating: {row["IMDB_Rating"]}")
                else:
                    row["IMDB_Rating"] = np.nan

            if pd.isna(row["Revenue"]) and "BoxOffice" in data:
                if data["BoxOffice"] != "N/A":
                    row["Revenue"] = data["BoxOffice"]
                    print(f"{title} -> Revenue: {row["Revenue"]}")
                else:
                    row["Revenue"] = np.nan

            if pd.isna(row["Genre"]) and "Genre" in data:
                if data["Genre"] != "N/A":
                    row["Genre"] = data["Genre"].split(",")[0].strip()
                    print(f"{title} -> Genre: {row["Genre"]}")
                else:
                    row["Genre"] = np.nan

        else:
            print(f"No encontrado en OMDb: {title}")
            
    else:
        print(f"Error con la API para {title}: {response.status_code}")

    return row

In [26]:
print("\nPorcentaje de nulos:")
display(round(df_movies.isnull().mean() * 100, 2))


Porcentaje de nulos:


Title           0.0
Genre          15.0
Year            0.0
Budget          0.0
Revenue        15.0
IMDB_Rating    20.0
dtype: float64

In [27]:
df_movies = df_movies.apply(fill_omdb, axis=1)

No encontrado en OMDb: Forgotten Realm
Digital Frontier -> Genre: Documentary
No encontrado en OMDb: Quantum Hearts
Beyond the Stars -> IMDB_Rating: 5.2
No encontrado en OMDb: Shattered Skies
No encontrado en OMDb: Eternal Voyage
Mystic River -> Genre: Crime


In [28]:
print("\nPorcentaje de nulos:")
display(round(df_movies.isnull().mean() * 100, 2))


Porcentaje de nulos:


Title           0.0
Genre           5.0
Year            0.0
Budget          0.0
Revenue        15.0
IMDB_Rating    15.0
dtype: float64

# 05. Imputación de nulos restantes
- Por ejemplo, Media, Mediana o valor constante

# 06. Exportar CSV limpio

In [19]:
df_movies

,Title,Genre,Year,Budget,Revenue,IMDB_Rating
0,The Last Journey,Drama,2020,11000000,3.404969e+07,4.9
1,Infinite Dreams,Drama,2004,178000000,5.504360e+08,6.6
2,Code of Shadows,Romance,2018,13000000,7.855773e+06,4.7
3,Ocean Whisper,Action,2016,148000000,1.769979e+08,7.2
4,Forgotten Realm,Romance,2017,61000000,1.264061e+08,NaN
5,Neon City,Comedy,2000,80000000,3.621845e+08,5.1
6,Silent Horizon,Fantasy,2006,31000000,2.556292e+07,4.1
7,Echoes of Tomorrow,Sci-Fi,2011,211000000,1.375882e+08,NaN
8,Digital Frontier,NaN,2003,25000000,6.080356e+07,8.2
9,Whispers in the Dark,Fantasy,2019,152000000,1.782975e+08,3.9


In [29]:
df_movies.to_csv("../data/clean.csv", index=False)